In [ ]:
from ipywidgets import interactive
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import seaborn as sns

from sem_image import SEM_image

In [ ]:
%matplotlib notebook

In [ ]:
BATCH = 'Batch 220301'


#load SEM data
filenames = []
for file in os.listdir('./data/' + BATCH + '/'):
    if file.endswith('.tif'):
        filenames.append(file[:-4])
filenames.sort()
SEM = {}
for name in tqdm(filenames):
    #SEM[name] = SEM_image('./'+name, Y_PXLS) # <- use this function to cut off scalebar (set Y_PXLS)
    SEM[name] = SEM_image('./data/' + BATCH + '/'+name)
    SEM[name].calculate_fft2D()
    SEM[name].calculate_radial_fft(N=500, R_min=6, R_max=12)
    SEM[name].fit_RDF()

In [ ]:
fig0,ax0 = plt.subplots()
def update_plot(name):
    SEM[name].show_image(fig0, ax0)
display(interactive(update_plot, name=filenames))

In [ ]:
fig1,ax1 = plt.subplots()
def update_plot(name):
    SEM[name].show_fft2D(fig1, ax1)
display(interactive(update_plot, name=filenames))

#For scaled FFT images, use the following function instead (slow):
#    SEM[name].show_fft2D_scaled(fig, ax)

In [ ]:
fig2,ax2 = plt.subplots()
def update_plot(i=0):
    name=filenames[i]
    SEM[name].show_RDF_fit(fig2,ax2)
    ax2.set_title(name)
display(interactive(update_plot, i=(0,len(filenames)-1)))

In [ ]:
#make groups
keys = []
for name in filenames:
    if name[:5] not in keys:
        keys.append(name[:5])
keys = np.sort(keys).tolist()

#define plot colors
palette = sns.color_palette('pastel')
def get_color(key):
    return palette[keys.index(key)]

#save data
data_organized = {}
for key in keys:
    data_organized[key] = {}
    for name in filenames:
        if name[:5] == key:
            data_organized[key][name] = SEM[name]


    
for key in keys:
    out = SEM[filenames[0]].fft1D_r
    rows = ['x-axis']
    for name in data_organized[key].keys():
        out = np.vstack((out, data_organized[key][name].fft1D))
        rows.append(name)
    out = np.vstack((rows, out.transpose()))
    
    with open('export/fft1D_'+key+'.txt', 'w') as f:
        np.savetxt(f, out, delimiter=', ', fmt='%s')

In [ ]:
#summarize results
results, avg_results, avg_sigma = {},{},{}
for key in keys:
    results[key] = []
    for name in filenames:
        if key == name[:5]:
            results[key].append(SEM[name].popt_RDF[0])
    avg_results[key] = np.nanmean(results[key])
    avg_sigma[key] = np.std(results[key])

In [ ]:
#plot results
fig3,ax3 = plt.subplots(figsize=(8,len(filenames)*0.3))

for i,name in enumerate(filenames):
    ax3.errorbar(SEM[name].popt_RDF[0], -i, xerr=SEM[name].popt_RDF[1], 
                label=SEM[name].name, color=get_color(SEM[name].name[:5]), marker='o', capsize=4)

ax3.minorticks_on()
ax3.grid(axis = 'x', which='both')
ax3.grid(axis = 'x', which='minor', linestyle=':')
ax3.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))
ax3.set_xlabel('radial distance [nm]')
ax3.set_yticks([])
fig3.tight_layout()

In [ ]:
#histogram
fig4,ax4 = plt.subplots(figsize=(8,3))
for key in keys:
    ax4.hist(results[key], color=get_color(key), label=key, alpha=0.3)

ax4.minorticks_on()
ax4.grid(axis = 'x', which='both')
ax4.grid(axis = 'x', which='minor', linestyle=':')
ax4.legend()
ax4.set_xlabel('radial distance [nm]')
fig4.tight_layout()

In [ ]:
#curve

fig5,ax5 = plt.subplots(figsize=(8,4))

ax5.plot([float(key.split('m')[0]) for key in keys], [avg_results[key] for key in keys], color='k')
for key in keys:
    ax5.errorbar(float(key.split('m')[0]), avg_results[key],yerr=avg_sigma[key],
                capsize=4, color=get_color(key), marker='s', label=key)
    
ax5.legend()
ax5.grid()
ax5.set_xlabel('ozonization time [min]')
ax5.set_ylabel('radial distance [nm]')
fig5.tight_layout()